# Data Cleaning and Feature Generation of Inflation Data


In [9]:
import pandas as pd
from pathlib import Path

Data-source: https://data-explorer.oecd.org/vis?tm=inflation&pg=0&snb=50&vw=tb&df%5Bds%5D=dsDisseminateFinalDMZ&df%5Bid%5D=DSD_PRICES%40DF_PRICES_HICP&df%5Bag%5D=OECD.SDD.TPS&df%5Bvs%5D=1.0&dq=HRV%2BBGR%2BTUR%2BGBR%2BCHE%2BSVN%2BSWE%2BESP%2BSVK%2BPRT%2BPOL%2BNOR%2BNLD%2BLUX%2BLTU%2BLVA%2BIRL%2BITA%2BISL%2BHUN%2BGRC%2BDEU%2BFRA%2BFIN%2BEST%2BDNK%2BBEL%2BCZE%2BAUT.M.HICP.CPI.PA._T.N.GY&to%5BTIME_PERIOD%5D=false&pd=2000-01%2C2024-10

In [10]:
df = pd.read_csv("../data/external/oecd-european-inflation-rates-since-2000.csv")
df.head()

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,REF_AREA,Reference area,FREQ,Frequency of observation,METHODOLOGY,Methodology,...,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,BASE_PER,Base period,DURABILITY,Durability,DECIMALS,Decimals
0,DATAFLOW,OECD.SDD.TPS:DSD_PRICES@DF_PRICES_HICP(1.0),Eurostat harmonised index of consumer prices (...,I,SVN,Slovenia,M,Monthly,HICP,Eurostat harmonised index of consumer prices (...,...,A,Normal value,NaN,NaN,NaN,NaN,NaN,NaN,2,Two
1,DATAFLOW,OECD.SDD.TPS:DSD_PRICES@DF_PRICES_HICP(1.0),Eurostat harmonised index of consumer prices (...,I,SVN,Slovenia,M,Monthly,HICP,Eurostat harmonised index of consumer prices (...,...,A,Normal value,NaN,NaN,NaN,NaN,NaN,NaN,2,Two
2,DATAFLOW,OECD.SDD.TPS:DSD_PRICES@DF_PRICES_HICP(1.0),Eurostat harmonised index of consumer prices (...,I,SVN,Slovenia,M,Monthly,HICP,Eurostat harmonised index of consumer prices (...,...,A,Normal value,NaN,NaN,NaN,NaN,NaN,NaN,2,Two
3,DATAFLOW,OECD.SDD.TPS:DSD_PRICES@DF_PRICES_HICP(1.0),Eurostat harmonised index of consumer prices (...,I,SVN,Slovenia,M,Monthly,HICP,Eurostat harmonised index of consumer prices (...,...,A,Normal value,NaN,NaN,NaN,NaN,NaN,NaN,2,Two
4,DATAFLOW,OECD.SDD.TPS:DSD_PRICES@DF_PRICES_HICP(1.0),Eurostat harmonised index of consumer prices (...,I,SVN,Slovenia,M,Monthly,HICP,Eurostat harmonised index of consumer prices (...,...,A,Normal value,NaN,NaN,NaN,NaN,NaN,NaN,2,Two


Store a file with all countries that we have data for:

In [12]:
df = df[["Reference area", "TIME_PERIOD", "OBS_VALUE"]]
df_countries = df.drop_duplicates(subset=["Reference area"])["Reference area"].to_frame()
Path("../data/interim/").mkdir(parents=True, exist_ok=True)
df_countries.to_feather("../data/interim/countries-with-inflation-data.ftr")

For Switzerland we only have inlfation data since December 2005 (see notebooks/0.02-lte-oced-european-inflation-rates-1.ipynb). We will handle this by removing all time periods for which we have missing data, as 226 data points should still be enough for our use case. Also, not all countries have reported inflation data for October 2024, so we will omit this data point too.

In [4]:
df_clean = df[(df["TIME_PERIOD"] >= "2005-12") & (df["TIME_PERIOD"] <= "2024-09")]

In [5]:
data_points_per_country = {}
for country in df_countries["Reference area"].tolist():
    df_country = df_clean[df_clean["Reference area"] == country]
    data_points_per_country[country] = len(df_country)

data_points_per_country = dict(sorted(data_points_per_country.items(), key=lambda item: item[1]))
data_points_per_country

{'Slovenia': 226,
 'Estonia': 226,
 'Denmark': 226,
 'Belgium': 226,
 'Luxembourg': 226,
 'Lithuania': 226,
 'Sweden': 226,
 'Poland': 226,
 'Netherlands': 226,
 'Austria': 226,
 'Norway': 226,
 'Ireland': 226,
 'Slovak Republic': 226,
 'Hungary': 226,
 'Latvia': 226,
 'Italy': 226,
 'United Kingdom': 226,
 'Greece': 226,
 'Switzerland': 226,
 'France': 226,
 'Iceland': 226,
 'Germany': 226,
 'Finland': 226,
 'Spain': 226,
 'Czechia': 226,
 'Türkiye': 226,
 'Portugal': 226,
 'Croatia': 226,
 'Bulgaria': 226}

In [6]:
df_clean.head()

,Reference area,TIME_PERIOD,OBS_VALUE
0,Slovenia,2020-08,-0.7
1,Slovenia,2020-07,-0.3
2,Slovenia,2020-06,-0.8
54,Slovenia,2009-07,-0.6
55,Slovenia,2009-06,0.2


In [7]:
df_clean.to_feather("../data/interim/inflation-data-clean.ftr")